TVAE Model
===========

In this guide we will go through a series of steps that will let you
discover functionalities of the `TVAE` model, including how to:

-   Create an instance of `TVAE`.
-   Fit the instance to your data.
-   Generate synthetic versions of your data.
-   Use `TVAE` to anonymize PII information.
-   Customize the data transformations to improve the learning process.
-   Specify hyperparameters to improve the output quality.

What is TVAE?
--------------

The `sdv.tabular.TVAE` model is based on the VAE-based Deep Learning
data synthesizer which was presented at the NeurIPS 2020 conference by
the paper titled [Modeling Tabular data using Conditional
GAN](https://arxiv.org/abs/1907.00503).

Let\'s now discover how to learn a dataset and later on generate
synthetic data with the same format and statistical properties by using
the `TVAE` class from SDV.

Quick Usage
-----------

We will start by loading one of our demo datasets, the
`student_placements`, which contains information about MBA students that
applied for placements during the year 2020.

<div class="alert alert-warning">

**Warning**

In order to follow this guide you need to have `tvae` installed on your
system. If you have not done it yet, please install `tvae` now by
executing the command `pip install sdv` in a terminal.

</div>

In [1]:
from sdv.demo import load_tabular_demo

data = load_tabular_demo('student_placements')
data.head()

,student_id,gender,second_perc,high_perc,high_spec,degree_perc,degree_type,work_experience,experience_years,employability_perc,mba_spec,mba_perc,salary,placed,start_date,end_date,duration
0,17264,M,67.00,91.00,Commerce,58.00,Sci&Tech,False,0,55.0,Mkt&HR,58.80,27000.0,True,2020-07-23,2020-10-12,3.0
1,17265,M,79.33,78.33,Science,77.48,Sci&Tech,True,1,86.5,Mkt&Fin,66.28,20000.0,True,2020-01-11,2020-04-09,3.0
2,17266,M,65.00,68.00,Arts,64.00,Comm&Mgmt,False,0,75.0,Mkt&Fin,57.80,25000.0,True,2020-01-26,2020-07-13,6.0
3,17267,M,56.00,52.00,Science,52.00,Sci&Tech,False,0,66.0,Mkt&HR,59.43,NaN,False,NaT,NaT,NaN
4,17268,M,85.80,73.60,Commerce,73.30,Comm&Mgmt,False,0,96.8,Mkt&Fin,55.50,42500.0,True,2020-07-04,2020-09-27,3.0


As you can see, this table contains information about students which
includes, among other things:

-   Their id and gender
-   Their grades and specializations
-   Their work experience
-   The salary that they where offered
-   The duration and dates of their placement

You will notice that there is data with the following characteristics:

-   There are float, integer, boolean, categorical and datetime values.
-   There are some variables that have missing data. In particular, all
    the data related to the placement details is missing in the rows
    where the student was not placed.

T   There are float, integer, boolean, categorical and datetime values.
-   There are some variables that have missing data. In particular, all
    the data related to the placement details is missing in the rows
    where the student was not placed.

Let us use `TVAE` to learn this data and then sample synthetic data
about new students to see how well de model captures the characteristics
indicated above. In order to do this you will need to:

-   Import the `sdv.tabular.TVAE` class and create an instance of it.
-   Call its `fit` method passing our table.
-   Call its `sample` method indicating the number of synthetic rows
    that you want to generate.

In [2]:
from sdv.tabular import TVAE

model = TVAE()
model.fit(data)

/Users/fealho/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:67: FutureWarning: Pass n_components=10 as keyword args. From version 0.25 passing these as positional arguments will result in an error
  warnings.warn("Pass {} as keyword args. From version 0.25 "
/Users/fealho/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:67: FutureWarning: Pass n_components=10 as keyword args. From version 0.25 passing these as positional arguments will result in an error
  warnings.warn("Pass {} as keyword args. From version 0.25 "
/Users/fealho/opt/anaconda3/lib/python3.8/site-packages/sklearn/mixture/_base.py:265: ConvergenceWarning: Initialization 1 did not converge. Try different init parameters, or increase max_iter, tol or check for degenerate data.
  warnings.warn('Initialization %d did not converge. '
/Users/fealho/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:67: FutureWarning: Pass n_components=10 as keyword args. From versi

<div class="alert alert-info">

**Note**

Notice that the model `fitting` process took care of transforming the
different fields using the appropriate [Reversible Data
Transforms](http://github.com/sdv-dev/RDT) to ensure that the data has a
format that the underlying TVAESynthesizer class can handle.

</div>

### Generate synthetic data from the model

Once the modeling has finished you are ready to generate new synthetic
data by calling the `sample` method from your model passing the number
of rows that we want to generate.

In [3]:
new_data = model.sample(200)

This will return a table identical to the one which the model was fitted
on, but filled with new data which resembles the original one.

In [4]:
new_data.head()

,student_id,gender,second_perc,high_perc,high_spec,degree_perc,degree_type,work_experience,experience_years,employability_perc,mba_spec,mba_perc,salary,placed,start_date,end_date,duration
0,17400,M,77.393985,67.849277,Science,63.241766,Comm&Mgmt,False,1,60.884922,Mkt&HR,56.406933,NaN,False,2020-03-04,NaT,6.0
1,17419,F,72.800354,70.661481,Arts,74.853185,Sci&Tech,True,1,64.703944,Mkt&Fin,58.249460,28790.627296,True,2020-01-19,NaT,3.0
2,17440,F,71.854794,64.496973,Arts,59.646384,Comm&Mgmt,True,0,67.158006,Mkt&HR,58.688960,28755.112015,True,2020-03-08,2020-10-08,3.0
3,17364,F,74.818015,63.806092,Arts,64.093333,Comm&Mgmt,True,1,63.983461,Mkt&HR,57.699225,NaN,False,2020-01-18,NaT,3.0
4,17442,F,73.451233,78.992041,Arts,62.144904,Sci&Tech,False,1,60.909026,Mkt&HR,59.394541,NaN,False,NaT,2020-12-12,12.0


<div class="alert alert-info">

**Note**

You can control the number of rows by specifying the number of `samples`
in the `model.sample(<num_rows>)`. To test, try `model.sample(10000)`.
Note that the original table only had \~200 rows.

</div>

### Save and Load the model

In many scenarios it will be convenient to generate synthetic versions
of your data directly in systems that do not have access to the original
data source. For example, if you may want to generate testing data on
the fly inside a testing environment that does not have access to your
production database. In these scenarios, fitting the model with real
data every time that you need to generate new data is feasible, so you
will need to fit a model in your production environment, save the fitted
model into a file, send this file to the testing environment and then
load it there to be able to `sample` from it.

Let\'s see how this process works.

#### Save and share the model

Once you have fitted the model, all you need to do is call its `save`
method passing the name of the file in which you want to save the model.
Note that the extension of the filename is not relevant, but we will be
using the `.pkl` extension to highlight that the serialization protocol
used is [pickle](https://docs.python.org/3/library/pickle.html).

In [5]:
model.save('my_model.pkl')

This will have created a file called `my_model.pkl` in the same
directory in which you are running SDV.

<div class="alert alert-info">

**Important**

If you inspect the generated file you will notice that its size is much
smaller than the size of the data that you used to generate it. This is
because the serialized model contains **no information about the
original data**, other than the parameters it needs to generate
synthetic versions of it. This means that you can safely share this
`my_model.pkl` file without the risc of disclosing any of your real
data!

</div>

#### Load the model and generate new data

The file you just generated can be send over to the system where the
synthetic data will be generated. Once it is there, you can load it
using the `TVAE.load` method, and then you are ready to sample new data
from the loaded instance:

In [6]:
loaded = TVAE.load('my_model.pkl')
new_data = loaded.sample(200)

<div class="alert alert-warning">

**Warning**

Notice that the system where the model is loaded needs to also have
`sdv` and `tvae` installed, otherwise it will not be able to load the
model and use it.

</div>

### Specifying the Primary Key of the table

One of the first things that you may have noticed when looking that demo
data is that there is a `student_id` column which acts as the primary
key of the table, and which is supposed to have unique values. Indeed,
if we look at the number of times that each value appears, we see that
all of them appear at most once:

In [7]:
data.student_id.value_counts().max()

1

However, if we look at the synthetic data that we generated, we observe
that there are some values that appear more than once:

In [8]:
new_data[new_data.student_id == new_data.student_id.value_counts().index[0]]

,student_id,gender,second_perc,high_perc,high_spec,degree_perc,degree_type,work_experience,experience_years,employability_perc,mba_spec,mba_perc,salary,placed,start_date,end_date,duration
10,17415,M,73.816167,68.774019,Arts,64.783188,Sci&Tech,False,1,61.659540,Mkt&HR,61.234688,NaN,False,NaT,NaT,6.0
12,17415,F,62.664495,61.405771,Arts,63.088248,Others,True,1,60.081593,Mkt&Fin,59.326543,NaN,True,2020-01-15,2020-10-03,3.0
22,17415,F,68.684677,76.598300,Arts,59.129584,Others,False,0,67.467540,Mkt&Fin,60.617534,27481.778035,False,2020-07-02,2020-09-29,6.0
48,17415,F,76.154368,69.433904,Arts,69.086352,Sci&Tech,False,0,57.219367,Mkt&Fin,59.508436,NaN,True,2020-07-19,2020-12-06,6.0
142,17415,F,57.174032,67.146382,Science,68.569899,Others,True,1,67.617152,Mkt&HR,59.505112,NaN,False,2020-01-18,NaT,6.0
164,17415,F,58.355590,60.814827,Arts,59.350694,Others,False,1,62.753040,Mkt&HR,59.384975,NaN,True,2020-03-06,NaT,3.0


This happens because the model was not notified at any point about the
fact that the `student_id` had to be unique, so when it generates new
data it will provoke collisions sooner or later. In order to solve this,
we can pass the argument `primary_key` to our model when we create it,
indicating the name of the column that is the index of the table.

In [9]:
model = TVAE(
    primary_key='student_id'
)
model.fit(data)
new_data = model.sample(200)
new_data.head()

/Users/fealho/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:67: FutureWarning: Pass n_components=10 as keyword args. From version 0.25 passing these as positional arguments will result in an error
  warnings.warn("Pass {} as keyword args. From version 0.25 "
/Users/fealho/opt/anaconda3/lib/python3.8/site-packages/sklearn/mixture/_base.py:265: ConvergenceWarning: Initialization 1 did not converge. Try different init parameters, or increase max_iter, tol or check for degenerate data.
  warnings.warn('Initialization %d did not converge. '
/Users/fealho/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:67: FutureWarning: Pass n_components=10 as keyword args. From version 0.25 passing these as positional arguments will result in an error
  warnings.warn("Pass {} as keyword args. From version 0.25 "
/Users/fealho/opt/anaconda3/lib/python3.8/site-packages/sklearn/mixture/_base.py:265: ConvergenceWarning: Initialization 1 did not converge. Try differ

,student_id,gender,second_perc,high_perc,high_spec,degree_perc,degree_type,work_experience,experience_years,employability_perc,mba_spec,mba_perc,salary,placed,start_date,end_date,duration
0,0,M,72.451138,70.190958,Arts,65.673292,Comm&Mgmt,False,2,72.581975,Mkt&HR,66.193222,75603.698962,True,NaT,2020-08-09,6.0
1,1,F,76.348514,81.448987,Arts,64.459611,Sci&Tech,False,1,63.639318,Mkt&Fin,58.924773,29481.704205,True,NaT,NaT,NaN
2,2,F,65.715928,82.804877,Arts,64.402524,Comm&Mgmt,False,1,62.635608,Mkt&HR,55.504409,29341.588149,True,NaT,NaT,6.0
3,3,F,71.770191,70.907518,Arts,62.580870,Comm&Mgmt,True,1,61.443080,Mkt&Fin,59.728497,NaN,True,NaT,2020-08-02,6.0
4,4,F,78.199462,78.984360,Science,63.545447,Comm&Mgmt,False,1,59.597350,Mkt&Fin,55.727859,NaN,True,NaT,2020-08-08,6.0


As a result, the model will learn that this column must be unique and
generate a unique sequence of values for the column:

In [10]:
new_data.student_id.value_counts().max()

1

### Anonymizing Personally Identifiable Information (PII)

There will be many cases where the data will contain Personally
Identifiable Information which we cannot disclose. In these cases, we
will want our Tabular Models to replace the information within these
fields with fake, simulated data that looks similar to the real one but
does not contain any of the original values.

Let\'s load a new dataset that contains a PII field, the
`student_placements_pii` demo, and try to generate synthetic versions of
it that do not contain any of the PII fields.

<div class="alert alert-info">

**Note**

The `student_placements_pii` dataset is a modified version of the
`student_placements` dataset with one new field, `address`, which
contains PII information about the students. Notice that this additional
`address` field has been simulated and does not correspond to data from
the real users.

</div>

In [11]:
data_pii = load_tabular_demo('student_placements_pii')
data_pii.head()

,student_id,address,gender,second_perc,high_perc,high_spec,degree_perc,degree_type,work_experience,experience_years,employability_perc,mba_spec,mba_perc,salary,placed,start_date,end_date,duration
0,17264,"70304 Baker Turnpike\nEricborough, MS 15086",M,67.00,91.00,Commerce,58.00,Sci&Tech,False,0,55.0,Mkt&HR,58.80,27000.0,True,2020-07-23,2020-10-12,3.0
1,17265,"805 Herrera Avenue Apt. 134\nMaryview, NJ 36510",M,79.33,78.33,Science,77.48,Sci&Tech,True,1,86.5,Mkt&Fin,66.28,20000.0,True,2020-01-11,2020-04-09,3.0
2,17266,"3702 Bradley Island\nNorth Victor, FL 12268",M,65.00,68.00,Arts,64.00,Comm&Mgmt,False,0,75.0,Mkt&Fin,57.80,25000.0,True,2020-01-26,2020-07-13,6.0
3,17267,Unit 0879 Box 3878\nDPO AP 42663,M,56.00,52.00,Science,52.00,Sci&Tech,False,0,66.0,Mkt&HR,59.43,NaN,False,NaT,NaT,NaN
4,17268,"96493 Kelly Canyon Apt. 145\nEast Steven, NC 3...",M,85.80,73.60,Commerce,73.30,Comm&Mgmt,False,0,96.8,Mkt&Fin,55.50,42500.0,True,2020-07-04,2020-09-27,3.0


If we use our tabular model on this new data we will see how the
synthetic data that it generates discloses the addresses from the real
students:

In [12]:
model = TVAE(
    primary_key='student_id',
)
model.fit(data_pii)
new_data_pii = model.sample(200)
new_data_pii.head()

/Users/fealho/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:67: FutureWarning: Pass n_components=10 as keyword args. From version 0.25 passing these as positional arguments will result in an error
  warnings.warn("Pass {} as keyword args. From version 0.25 "
/Users/fealho/opt/anaconda3/lib/python3.8/site-packages/sklearn/mixture/_base.py:265: ConvergenceWarning: Initialization 1 did not converge. Try different init parameters, or increase max_iter, tol or check for degenerate data.
  warnings.warn('Initialization %d did not converge. '
/Users/fealho/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:67: FutureWarning: Pass n_components=10 as keyword args. From version 0.25 passing these as positional arguments will result in an error
  warnings.warn("Pass {} as keyword args. From version 0.25 "
/Users/fealho/opt/anaconda3/lib/python3.8/site-packages/sklearn/mixture/_base.py:265: ConvergenceWarning: Initialization 1 did not converge. Try differ

,student_id,address,gender,second_perc,high_perc,high_spec,degree_perc,degree_type,work_experience,experience_years,employability_perc,mba_spec,mba_perc,salary,placed,start_date,end_date,duration
0,0,"88421 Medina Prairie Suite 001\nPort Amanda, N...",M,76.459976,64.063517,Commerce,71.342844,Sci&Tech,False,1,70.442819,Mkt&Fin,65.799307,28071.830220,True,2020-01-19,2020-04-04,NaN
1,1,"2120 Kelsey Glens Apt. 004\nNew Markland, CA 7...",M,63.335002,61.534600,Arts,67.440754,Sci&Tech,False,0,79.737670,Mkt&Fin,65.208955,47314.221167,False,NaT,NaT,NaN
2,2,"9518 Goodman Station Suite 019\nWest Charles, ...",M,57.333477,60.795477,Commerce,70.857731,Comm&Mgmt,False,0,74.779541,Mkt&Fin,67.704226,40369.161326,False,2020-01-20,2020-09-09,NaN
3,3,"2120 Kelsey Glens Apt. 004\nNew Markland, CA 7...",M,53.632125,60.906503,Commerce,65.593969,Others,False,0,78.634661,Mkt&HR,62.105505,NaN,True,2020-01-19,2020-03-30,NaN
4,4,"2707 Maria Parkways Apt. 743\nAlisonview, TN 8...",M,62.797838,63.373926,Commerce,72.807120,Comm&Mgmt,False,0,68.733556,Mkt&Fin,59.444909,52813.622079,True,2020-01-28,2020-10-16,NaN


More specifically, we can see how all the addresses that have been
generated actually come from the original dataset:

In [13]:
new_data_pii.address.isin(data_pii.address).sum()

200

In order to solve this, we can pass an additional argument
`anonymize_fields` to our model when we create the instance. This
`anonymize_fields` argument will need to be a dictionary that contains:

-   The name of the field that we want to anonymize.
-   The category of the field that we want to use when we generate fake
    values for it.

The list complete list of possible categories can be seen in the [Faker
Providers](https://faker.readthedocs.io/en/master/providers.html) page,
and it contains a huge list of concepts such as:

-   name
-   address
-   country
-   city
-   ssn
-   credit_card_number
-   credit_card_expire
-   credit_card_security_code
-   email
-   telephone
-   \...

In this case, since the field is an e-mail address, we will pass a
dictionary indicating the category `address`

In [14]:
model = TVAE(
    primary_key='student_id',
    anonymize_fields={
        'address': 'address'
    }
)
model.fit(data_pii)

/Users/fealho/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:67: FutureWarning: Pass n_components=10 as keyword args. From version 0.25 passing these as positional arguments will result in an error
  warnings.warn("Pass {} as keyword args. From version 0.25 "
/Users/fealho/opt/anaconda3/lib/python3.8/site-packages/sklearn/mixture/_base.py:265: ConvergenceWarning: Initialization 1 did not converge. Try different init parameters, or increase max_iter, tol or check for degenerate data.
  warnings.warn('Initialization %d did not converge. '
/Users/fealho/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:67: FutureWarning: Pass n_components=10 as keyword args. From version 0.25 passing these as positional arguments will result in an error
  warnings.warn("Pass {} as keyword args. From version 0.25 "
/Users/fealho/opt/anaconda3/lib/python3.8/site-packages/sklearn/mixture/_base.py:265: ConvergenceWarning: Initialization 1 did not converge. Try differ

As a result, we can see how the real `address` values have been replaced
by other fake addresses:

In [15]:
new_data_pii = model.sample(200)
new_data_pii.head()

,student_id,address,gender,second_perc,high_perc,high_spec,degree_perc,degree_type,work_experience,experience_years,employability_perc,mba_spec,mba_perc,salary,placed,start_date,end_date,duration
0,0,USNS Woods\nFPO AP 06427,M,68.229966,69.812226,Commerce,57.824980,Sci&Tech,True,1,69.172674,Mkt&Fin,62.124233,NaN,True,2020-01-17,2020-09-27,6.0
1,1,USS Hughes\nFPO AP 02787,M,78.996027,66.108772,Commerce,56.710947,Sci&Tech,True,1,54.649897,Mkt&HR,56.816735,NaN,True,2020-03-03,NaT,NaN
2,2,"16166 Tara Flats Apt. 642\nLindaside, CT 64397",F,85.515743,54.096726,Commerce,56.938298,Sci&Tech,True,1,66.840977,Mkt&Fin,59.362912,NaN,True,NaT,2020-10-10,NaN
3,3,"679 Sanchez Plain Apt. 584\nMichaelborough, ME...",F,73.884908,80.789094,Commerce,55.524546,Sci&Tech,True,1,65.227241,Mkt&HR,72.665343,NaN,True,2020-03-01,2020-11-01,6.0
4,4,"6368 Gerald Alley\nNew Nicholasmouth, ME 90569",F,74.074378,63.755400,Commerce,59.073905,Sci&Tech,True,1,72.751680,Mkt&HR,60.688326,NaN,True,NaT,NaT,6.0


Which means that none of the original addresses can be found in the
sampled data:

In [16]:
data_pii.address.isin(new_data_pii.address).sum()

0

As we can see, in this case these modifications changed the obtained
results slightly, but they did neither introduce dramatic changes in the
performance.

### Conditional Sampling

As the name implies, conditional sampling allows us to sample from a conditional distribution using the `TVAE` model, which means we can generate only values that satisfy certain conditions. These conditional values can be passed to the `conditions` parameter in the `sample` method either as a dataframe or a dictionary.

In case a dictionary is passed, the model will generate as many rows as requested, all of which will satisfy the specified conditions, such as `gender = M`.

In [17]:
conditions = {
    "gender": 'M'
}
model.sample(5, conditions=conditions)

,student_id,address,gender,second_perc,high_perc,high_spec,degree_perc,degree_type,work_experience,experience_years,employability_perc,mba_spec,mba_perc,salary,placed,start_date,end_date,duration
0,2,"1888 Sanchez Streets Apt. 559\nClarkton, IA 43304",M,78.793720,66.651796,Commerce,59.368034,Sci&Tech,True,1,60.335075,Mkt&Fin,68.536610,NaN,True,NaT,NaT,3.0
1,4,"7046 Glover Garden Suite 170\nTerryfurt, KS 12002",M,68.722264,60.414743,Commerce,61.754384,Sci&Tech,True,1,78.570250,Mkt&Fin,66.283778,NaN,False,NaT,2020-09-28,6.0
2,0,"69782 Traci Shore\nAmandastad, IN 20814",M,76.960674,66.354372,Arts,53.606778,Sci&Tech,True,1,58.925662,Mkt&Fin,69.784011,NaN,True,NaT,NaT,6.0
3,1,"008 Deborah Loop Apt. 120\nJacksonhaven, WA 52383",M,76.313658,57.927821,Commerce,58.628391,Sci&Tech,True,2,59.870024,Mkt&Fin,68.356303,NaN,True,NaT,NaT,6.0
4,2,"8229 Heather Turnpike\nAtkinsonstad, FL 01951",M,74.348653,62.672687,Commerce,62.965027,Sci&Tech,True,1,57.439776,Mkt&Fin,65.451031,NaN,True,NaT,NaT,6.0


It's also possible to condition on multiple columns, such as `gender = M, work_experience = False, high_spec = Science`.

In [18]:
conditions = {
    'gender': 'M',
    'work_experience': False,
    'high_spec': 'Science'
}
model.sample(5, conditions=conditions)

,student_id,address,gender,second_perc,high_perc,high_spec,degree_perc,degree_type,work_experience,experience_years,employability_perc,mba_spec,mba_perc,salary,placed,start_date,end_date,duration
0,6,"707 Cantrell Squares\nLake Ashley, MT 05239",M,77.341289,62.845356,Science,58.144923,Sci&Tech,True,1,56.691474,Mkt&Fin,66.146952,NaN,True,NaT,2020-12-16,6.0
1,14,"6737 Martin Extension\nWest Meaganstad, MO 89703",M,74.261756,63.362602,Science,57.453795,Sci&Tech,True,1,58.071526,Mkt&Fin,59.943444,NaN,True,NaT,2020-08-12,6.0
2,17,"6845 Parker Islands Apt. 561\nLake Monique, PA...",M,77.396128,67.069388,Science,64.550860,Sci&Tech,True,0,60.567930,Mkt&Fin,63.271367,NaN,True,NaT,NaT,6.0
3,21,"747 Blackwell Flats Suite 126\nLake Brad, MS 6...",M,63.289159,68.053608,Science,52.533989,Sci&Tech,True,1,58.157778,Mkt&Fin,66.580605,NaN,True,2020-01-18,2020-12-13,NaN
4,0,"81144 Dunn Falls\nLake Carlymouth, NJ 21821",M,69.919969,63.473503,Science,56.277767,Sci&Tech,True,1,76.740107,Mkt&Fin,65.819823,NaN,True,NaT,2021-01-16,6.0


`conditions` can also be passed as a dataframe. In that case, the model will generate one sample for each row of the dataframe, sorted in the same order. Since the model already knows how many samples to generate, passing it as a parameter is unnecessary. For example, if we want to generate three samples where `gender = M` and three samples with `gender = F`, we can do the following: 

In [19]:
import pandas as pd 

conditions = pd.DataFrame({
    'gender': ['M', 'M', 'M', 'F', 'F', 'F'],
    'work_experience': [True, True, True, True, True, True]
})
model.sample(conditions=conditions)

,student_id,address,gender,second_perc,high_perc,high_spec,degree_perc,degree_type,work_experience,experience_years,employability_perc,mba_spec,mba_perc,salary,placed,start_date,end_date,duration
0,1,"0936 Miller Key Apt. 398\nLake Adrianchester, ...",M,72.627202,65.054335,Commerce,57.761051,Sci&Tech,True,1,55.970711,Mkt&Fin,64.465255,NaN,True,NaT,NaT,6.0
1,2,"675 Deanna Grove Apt. 718\nEast Deannaton, TN ...",M,74.197075,46.673379,Commerce,63.718591,Sci&Tech,True,1,58.519815,Mkt&Fin,63.210574,NaN,True,NaT,2020-09-03,6.0
2,3,"7307 Hannah Extension\nSmithfurt, MS 01578",M,76.758681,64.455877,Commerce,58.009777,Sci&Tech,True,1,64.536643,Mkt&Fin,66.147456,NaN,True,NaT,2020-08-27,6.0
3,2,"473 Elizabeth Point\nWalkerton, WI 63643",F,74.988042,57.318895,Commerce,63.716204,Sci&Tech,True,2,62.053636,Mkt&Fin,71.241303,NaN,True,NaT,NaT,6.0
4,0,"9690 Kayla Ramp\nStaffordmouth, TX 06562",F,74.001429,66.098761,Commerce,59.502307,Sci&Tech,True,1,60.607185,Mkt&Fin,62.297512,NaN,True,NaT,2021-01-03,6.0
5,3,"59612 Miller Parkways\nPaigefurt, NM 06322",F,68.132315,67.185858,Commerce,70.926619,Sci&Tech,True,2,55.172893,Mkt&Fin,54.941861,NaN,True,NaT,NaT,6.0


`TVAE` also supports conditioning on continuous values, as long as the values are within the range of seen numbers. For example, if all the values of the dataset are within 0 and 1, `TVAE` will not be able to set this value to 1000.

In [20]:
conditions = {
    'degree_perc': 70.0
}
model.sample(5, conditions=conditions)

/Users/fealho/Desktop/SDV/sdv/tabular/base.py:163: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sampled[column] = value
/Users/fealho/Desktop/SDV/sdv/tabular/base.py:163: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sampled[column] = value
/Users/fealho/Desktop/SDV/sdv/tabular/base.py:163: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas

,student_id,address,gender,second_perc,high_perc,high_spec,degree_perc,degree_type,work_experience,experience_years,employability_perc,mba_spec,mba_perc,salary,placed,start_date,end_date,duration
0,8,USS Zimmerman\nFPO AP 37283,M,59.043764,64.191863,Commerce,70.0,Sci&Tech,True,1,53.277168,Mkt&Fin,70.832009,NaN,True,2020-01-04,2020-10-07,NaN
1,8,"09456 Scott Walks Apt. 904\nNorth Donna, IA 81535",F,76.382905,61.410084,Commerce,70.0,Sci&Tech,True,1,65.044886,Mkt&Fin,56.483514,NaN,False,NaT,2020-11-13,6.0
2,0,2837 Kristin Curve Suite 298\nNorth Karenville...,F,76.361257,73.607689,Commerce,70.0,Sci&Tech,True,2,58.753069,Mkt&Fin,65.003135,NaN,True,NaT,NaT,6.0
3,3,"019 Gonzales Villages Suite 949\nPort Sara, NC...",M,66.434043,64.312919,Commerce,70.0,Sci&Tech,False,1,57.219812,Mkt&Fin,67.120759,NaN,True,NaT,NaT,6.0
4,32,"0572 Kevin Glen\nSmithbury, IL 42397",F,59.349983,69.966140,Commerce,70.0,Sci&Tech,True,2,54.273903,Mkt&Fin,65.093300,NaN,True,2020-03-02,NaT,6.0


<div class="alert alert-info">

**Note**
Currently, conditional sampling works through a rejection sampling process, where rows are sampled repeatedly until one that satisfies the conditions is found. In case you are running into a `Could not get enough valid rows within x trials` or simply wish to optimize the results, there are three parameters that can be fine-tuned: max_rows_multiplier, max_retries and float_rtol. More information about these parameters can be found in the API section.


</div>

### How do I specify constraints?

If you look closely at the data you may notice that some properties were
not completely captured by the model. For example, you may have seen
that sometimes the model produces an `experience_years` number greater
than `0` while also indicating that `work_experience` is `False`. These
type of properties are what we call `Constraints` and can also be
handled using `SDV`. For further details about them please visit the
[Handling Constraints](04_Handling_Constraints.ipynb) guide.

### Can I evaluate the Synthetic Data?

A very common question when someone starts using **SDV** to generate
synthetic data is: *\"How good is the data that I just generated?\"*

In order to answer this question, **SDV** has a collection of metrics
and tools that allow you to compare the *real* that you provided and the
*synthetic* data that you generated using **SDV** or any other tool.

You can read more about this in the [Evaluating Synthetic Data Generators](
05_Evaluating_Synthetic_Data_Generators.ipynb) guide.